In [114]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score , accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor , RandomForestRegressor
from warnings import filterwarnings
filterwarnings('ignore')
data = pd.read_csv("CarPrice_Assignment.csv")
df = data.copy()
df = df.iloc[:,1:]
df.head()

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   symboling         205 non-null    int64  
 1   CarName           205 non-null    object 
 2   fueltype          205 non-null    object 
 3   aspiration        205 non-null    object 
 4   doornumber        205 non-null    object 
 5   carbody           205 non-null    object 
 6   drivewheel        205 non-null    object 
 7   enginelocation    205 non-null    object 
 8   wheelbase         205 non-null    float64
 9   carlength         205 non-null    float64
 10  carwidth          205 non-null    float64
 11  carheight         205 non-null    float64
 12  curbweight        205 non-null    int64  
 13  enginetype        205 non-null    object 
 14  cylindernumber    205 non-null    object 
 15  enginesize        205 non-null    int64  
 16  fuelsystem        205 non-null    object 
 1

In [116]:
dums = pd.get_dummies(df[["CarName" , "fueltype" , "aspiration" , "carbody" , "drivewheel" ,
                          "enginelocation" , "enginetype" , "fuelsystem" ]])
y = df["price"]
X_ = df.drop(["price" , "CarName" , "fueltype" , "aspiration" , "carbody" , 
              "drivewheel" , "enginelocation" , "enginetype" , "fuelsystem"] , axis = 1)
X = pd.concat([X_ , dums] , axis = 1)
X["doornumber"] = X["doornumber"].replace({"two": 2, "four": 4})
X['cylindernumber'] = X['cylindernumber'].replace({'four' : 4 , 'six' : 6 ,'five' : 5 ,
                                                   'three' : 3,'twelve' : 12 , 'two' : 2 ,
                                                   'eight' : 8})
X.describe()

,symboling,doornumber,wheelbase,carlength,carwidth,carheight,curbweight,cylindernumber,enginesize,boreratio,...,enginetype_ohcv,enginetype_rotor,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,...,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.834146,3.121951,98.756585,174.049268,65.907805,53.724878,2555.565854,4.380488,126.907317,3.329756,...,0.063415,0.019512,0.053659,0.321951,0.014634,0.097561,0.004878,0.458537,0.043902,0.004878
std,1.245307,0.994966,6.021776,12.337289,2.145204,2.443522,520.680204,1.080854,41.642693,0.270844,...,0.244304,0.138655,0.225894,0.468368,0.120377,0.297446,0.069843,0.499498,0.205380,0.069843
min,-2.000000,2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,2.000000,61.000000,2.540000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,4.000000,97.000000,3.150000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,4.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,4.000000,120.000000,3.310000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,4.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,4.000000,141.000000,3.580000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,3.000000,4.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,12.000000,326.000000,3.940000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [117]:
y.describe()

count      205.000000
mean     13276.710571
std       7988.852332
min       5118.000000
25%       7788.000000
50%      10295.000000
75%      16503.000000
max      45400.000000
Name: price, dtype: float64

# KNN

In [118]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.25 , 
                                                       random_state = 144)
knn_model = KNeighborsRegressor().fit(X_train , y_train)

In [6]:
knn_model.n_neighbors

5

In [7]:
knn_model.effective_metric_

'euclidean'

In [8]:
y_pred = knn_model.predict(X_test)

In [9]:
np.sqrt(mean_squared_error(y_test , y_pred))

2581.275324725503

In [10]:
RMSE = [] 

for k in range(20):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_train) 
    rmse = np.sqrt(mean_squared_error(y_train,y_pred)) 
    RMSE.append(rmse) 
    print("k =" , k , "için RMSE değeri: ", rmse)

k = 1 için RMSE değeri:  0.0
k = 2 için RMSE değeri:  1848.0338536230095
k = 3 için RMSE değeri:  2338.754951176145
k = 4 için RMSE değeri:  2633.997724303632
k = 5 için RMSE değeri:  2891.2482958852725
k = 6 için RMSE değeri:  3104.0309593416528
k = 7 için RMSE değeri:  3260.259278746216
k = 8 için RMSE değeri:  3386.0036089698665
k = 9 için RMSE değeri:  3554.0438372173203
k = 10 için RMSE değeri:  3676.673051384621
k = 11 için RMSE değeri:  3828.6979731778697
k = 12 için RMSE değeri:  3932.6507712743637
k = 13 için RMSE değeri:  4093.713857200639
k = 14 için RMSE değeri:  4274.94399035793
k = 15 için RMSE değeri:  4347.44112732371
k = 16 için RMSE değeri:  4341.927447386445
k = 17 için RMSE değeri:  4413.4033195085485
k = 18 için RMSE değeri:  4529.522898749363
k = 19 için RMSE değeri:  4538.0829566276225
k = 20 için RMSE değeri:  4621.130442881232


## Model Tuning

In [11]:
knn_params = {'n_neighbors': np.arange(1,20,1)}

In [12]:
knn = KNeighborsRegressor()

In [13]:
knn_cv_model = GridSearchCV(knn, knn_params, cv = 10)

In [14]:
knn_cv_model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])})

In [15]:
print("Best parameters: ", knn_cv_model.best_params_)

Best parameters:  {'n_neighbors': 3}


In [16]:
RMSE = [] 
RMSE_CV = []
for k in range(20):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_train) 
    rmse = np.sqrt(mean_squared_error(y_train,y_pred)) 
    rmse_cv = np.sqrt(-1*cross_val_score(knn_model, X_train, y_train, cv=10, 
                                         scoring = "neg_mean_squared_error").mean())
    RMSE.append(rmse) 
    RMSE_CV.append(rmse_cv)
    print("k =" , k , "için RMSE değeri: ", rmse, "RMSE_CV değeri: ", rmse_cv )

k = 1 için RMSE değeri:  0.0 RMSE_CV değeri:  3933.32860206195
k = 2 için RMSE değeri:  1848.0338536230095 RMSE_CV değeri:  3494.5538779044828
k = 3 için RMSE değeri:  2338.754951176145 RMSE_CV değeri:  3484.171937785813
k = 4 için RMSE değeri:  2633.997724303632 RMSE_CV değeri:  3615.333390004152
k = 5 için RMSE değeri:  2891.2482958852725 RMSE_CV değeri:  3688.6346664240273
k = 6 için RMSE değeri:  3104.0309593416528 RMSE_CV değeri:  3751.886433288059
k = 7 için RMSE değeri:  3260.259278746216 RMSE_CV değeri:  3834.3815950957483
k = 8 için RMSE değeri:  3386.0036089698665 RMSE_CV değeri:  3964.831910000935
k = 9 için RMSE değeri:  3554.0438372173203 RMSE_CV değeri:  4120.569303004406
k = 10 için RMSE değeri:  3676.673051384621 RMSE_CV değeri:  4274.537947610628
k = 11 için RMSE değeri:  3828.6979731778697 RMSE_CV değeri:  4392.230553097674
k = 12 için RMSE değeri:  3932.6507712743637 RMSE_CV değeri:  4538.529007690019
k = 13 için RMSE değeri:  4093.713857200639 RMSE_CV değeri:  4600.

In [17]:
knn_tuned = KNeighborsRegressor(n_neighbors = knn_cv_model.best_params_["n_neighbors"])

In [18]:
knn_tuned.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [19]:
np.sqrt(mean_squared_error(y_test, knn_tuned.predict(X_test)))

2451.130007555685

# SVR

In [20]:
from sklearn.svm import SVR

In [21]:
svr_model = SVR("rbf").fit(X_train , y_train)

In [22]:
y_pred = svr_model.predict(X_test)

In [23]:
np.sqrt(mean_squared_error(y_test , y_pred))

7005.097288178883

## Model Tuning

In [24]:
svr_params = {"C" : [0.01 , 0.1 , 0.4 , 5 , 10 , 20 , 30 , 40, 50]}
svr_cv_model = GridSearchCV(svr_model , svr_params , cv = 10)
svr_cv_model.fit(X_train , y_train)

GridSearchCV(cv=10, estimator=SVR(),
             param_grid={'C': [0.01, 0.1, 0.4, 5, 10, 20, 30, 40, 50]})

In [25]:
pd.Series(svr_cv_model.best_params_)[0]

50

In [26]:
svr_tuned = SVR("rbf" , C = pd.Series(svr_cv_model.best_params_)[0]).fit(X_train , y_train)

In [27]:
y_pred = svr_tuned.predict(X_test)

In [28]:
np.sqrt(mean_squared_error(y_test , y_pred))

6935.648036872986

# Neural Networks

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [30]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [31]:
from sklearn.neural_network import MLPRegressor

In [32]:
mlp_model = MLPRegressor(hidden_layer_sizes = (100,20)).fit(X_train_scaled , y_train )

In [33]:
y_pred = mlp_model.predict(X_test_scaled)

In [34]:
np.sqrt(mean_squared_error(y_test , y_pred))

13198.230300322171

## Model Tuning

In [35]:
mlp_model

MLPRegressor(hidden_layer_sizes=(100, 20))

In [36]:
mlp_params = {'alpha': [0.1, 0.01,0.02,0.005],
             'hidden_layer_sizes': [(20,20),(100,50,150),(300,200,150)],
             'activation': ['relu','logistic']}

In [37]:
mlp_cv_model = GridSearchCV(mlp_model , mlp_params , cv = 10 , n_jobs = -1)

In [38]:
mlp_cv_model.fit(X_train_scaled , y_train)

GridSearchCV(cv=10, estimator=MLPRegressor(hidden_layer_sizes=(100, 20)),
             n_jobs=-1,
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005],
                         'hidden_layer_sizes': [(20, 20), (100, 50, 150),
                                                (300, 200, 150)]})

In [39]:
mlp_cv_model.best_params_

{'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (300, 200, 150)}

In [40]:
mlp_tuned = MLPRegressor(alpha = 0.01 , hidden_layer_sizes = (300, 200, 150) , activation = 'relu')

In [41]:
mlp_tuned.fit(X_train_scaled , y_train)

MLPRegressor(alpha=0.01, hidden_layer_sizes=(300, 200, 150))

In [42]:
y_pred = mlp_tuned.predict(X_test_scaled)

In [43]:
np.sqrt(mean_squared_error(y_test , y_pred))

3801.8644192839993

# CART

In [46]:
cart_model = DecisionTreeRegressor()

In [47]:
cart_model.fit(X_train , y_train)

DecisionTreeRegressor()

In [50]:
!pip install skompiler

  Created wheel for skompiler: filename=SKompiler-0.7-py3-none-any.whl size=54316 sha256=478a52624c23a17d70482387588103a76e4f9cbcb3a913ce2fd395b99549fbb9
  Stored in directory: c:\users\karacik\appdata\local\pip\cache\wheels\eb\73\c6\cf0fc98d965e01e8bb3b1923d1e6a57374645d5d055f00e898
Successfully built skompiler


In [53]:
!pip install astor

In [51]:
from skompiler import skompile

In [54]:
print(skompile(cart_model.predict).to('python/code'))

(((((((((((6488.0 if x[6] <= 2027.5 else 6338.0) if x[4] <= 
    63.69999885559082 else 6377.0 if x[6] <= 1890.5 else ((6695.0 if x[11] <=
    9.199999809265137 else 6692.0) if x[84] <= 0.5 else 6669.0) if x[77] <=
    0.5 else 6795.0) if x[9] <= 3.0649999380111694 else 7395.0) if x[44] <=
    0.5 else 7609.0) if x[15] <= 39.5 else ((5399.0 if x[9] <= 
    2.940000057220459 else 5572.0) if x[14] <= 42.5 else 5151.0) if x[10] <=
    3.3200000524520874 else 6479.0) if x[12] <= 68.5 else 8916.5 if x[8] <=
    91.0 else ((((6855.0 if x[6] <= 1878.5 else 6849.0) if x[97] <= 0.5 else
    6649.0) if x[53] <= 0.5 else 6529.0) if x[16] <= 0.5 else 5499.0) if x[
    6] <= 1945.5 else (((7198.0 if x[10] <= 3.2200000286102295 else 7129.0) if
    x[5] <= 52.89999961853027 else 6938.0 if x[169] <= 0.5 else 7053.0) if 
    x[2] <= 96.0999984741211 else 7295.0) if x[14] <= 30.5 else 8249.0 if x
    [3] <= 163.8499984741211 else ((7299.0 if x[6] <= 1987.5 else 7349.0) if
    x[92] <= 0.5 else 7499.0) i

In [56]:
cart_model.predict(X_train)[0:5]

array([15645., 11850.,  9233., 17950., 15040.])

In [59]:
y_test[0:5]

136    18150.0
23      7957.0
75     16503.0
57     13645.0
195    13415.0
Name: price, dtype: float64

In [62]:
y_pred = cart_model.predict(X_test)

In [63]:
np.sqrt(mean_squared_error(y_test , y_pred))

2348.723255875904

# Model Tuning

In [65]:
?cart_model

Type:        DecisionTreeRegressor
String form: DecisionTreeRegressor()
File:        c:\users\karacik\anaconda3\lib\site-packages\sklearn\tree\_classes.py
Docstring:  
A decision tree regressor.

Read more in the :ref:`User Guide <tree>`.

Parameters
----------
criterion : {"mse", "friedman_mse", "mae"}, default="mse"
    The function to measure the quality of a split. Supported criteria
    are "mse" for the mean squared error, which is equal to variance
    reduction as feature selection criterion and minimizes the L2 loss
    using the mean of each terminal node, "friedman_mse", which uses mean
    squared error with Friedman's improvement score for potential splits,
    and "mae" for the mean absolute error, which minimizes the L1 loss
    using the median of each terminal node.

    .. versionadded:: 0.18
       Mean Absolute Error (MAE) criterion.

splitter : {"best", "random"}, default="best"
    The strategy used to choose the split at each node. Supported
    strategies are "b

In [66]:
cart_params = {"min_samples_split": range(2,100),
               "max_leaf_nodes": range(2,10)}

In [67]:
cart_cv_model = GridSearchCV(cart_model , cart_params , cv = 10)

In [68]:
cart_cv_model.fit(X_train , y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_leaf_nodes': range(2, 10),
                         'min_samples_split': range(2, 100)})

In [69]:
cart_cv_model.best_params_

{'max_leaf_nodes': 9, 'min_samples_split': 25}

In [70]:
cart_tuned = DecisionTreeRegressor(max_leaf_nodes = 9 , min_samples_split = 25)

In [71]:
cart_tuned.fit(X_train , y_train)

DecisionTreeRegressor(max_leaf_nodes=9, min_samples_split=25)

In [72]:
y_pred = cart_tuned.predict(X_test)

In [73]:
np.sqrt(mean_squared_error(y_test , y_pred))

2537.027464742795

# Bagged Trees Regression

In [74]:
bag_model = BaggingRegressor(bootstrap_features = True)

In [75]:
bag_model.fit(X_train , y_train)

BaggingRegressor(bootstrap_features=True)

In [77]:
bag_model.n_estimators

10

In [78]:
bag_model.estimators_

[DecisionTreeRegressor(random_state=2003811152),
 DecisionTreeRegressor(random_state=1113451934),
 DecisionTreeRegressor(random_state=597668642),
 DecisionTreeRegressor(random_state=923672613),
 DecisionTreeRegressor(random_state=849275828),
 DecisionTreeRegressor(random_state=1862903350),
 DecisionTreeRegressor(random_state=614998131),
 DecisionTreeRegressor(random_state=358471317),
 DecisionTreeRegressor(random_state=1787766595),
 DecisionTreeRegressor(random_state=764238339)]

In [79]:
bag_model.estimators_samples_

[array([  2, 123, 122, 131,  20, 141,   0, 149,  30,  95,  77,  59, 119,
        138, 119, 102,  35,  71,  61, 139,  57,  39, 107,  32,  95, 129,
        144, 105,  95,  86, 119, 109,  84,  86, 118,  48,  52,  90,  71,
        102,  32,  85, 129, 113,  94,  19, 152, 105, 133,  95, 127,  42,
         75,  42,  54,  30,  45, 107, 150,  90,  62, 135,  77,  37,  44,
         28, 138,  88, 144, 117, 128, 122, 115,  60,   8,  45,  29,  85,
         17, 123,  68, 102,  41,  65,  25, 137, 122,   8,  22, 152,  19,
        113, 113, 130,  11, 104,  52,  88,  62,  57,  80,  45,  43,  14,
         11, 122, 101, 114, 114, 119, 118,  55,  23,  32,  84,  33, 104,
         78,  63,  40,  13,  97,  73, 100, 113, 134, 139,  60, 104,  47,
        108,  62,  64,  12,  77, 124, 114,  26, 111,  56, 111, 112,  47,
         30,  63, 137,  53,  85, 142,  18,  68, 135,  45]),
 array([ 21,   6,  40,  18,  87,  73,  46, 114,  50, 125,  37,  57,  35,
         78, 143,  19,  66,  32,  16,   0,  99,  20, 137,  30, 1

In [80]:
bag_model.estimators_features_

[array([ 80, 139, 125, 177,  70, 147, 163, 166,  36,  74, 122, 145, 113,
          9, 130, 125,   5,   4,  63,  23, 162,  23,  47,  97,  19,  45,
        112, 114, 101,  14, 189,  90,  68, 168,  92, 170, 130,  75, 108,
        110,  53, 114,  73, 150,  44,  26, 146,  74,  16,  41, 152,  61,
        144,  81,  76,  83, 191, 121, 141,   8,  47, 114, 135,  18,  91,
         81,  38,  93, 121, 183, 171,  91,  58, 129,  24,  91,  20, 180,
         76,  68,  42, 170, 113, 183,  16, 124, 108, 146, 187, 131, 182,
         67, 131,  41,  62,   8, 150, 183,  23,  83,  10,  32, 156,  26,
        107, 134, 133,  23, 172,  58, 101, 116, 173,  82,  84, 156,  96,
         42,  28,  83, 132, 167,  57,  34,  26, 171,  39,  69, 118,  46,
        134,   7, 150,  82, 108, 156,  91,  12, 180,  61,  48, 101,  37,
          9,  32,  89, 100, 187, 113, 161,   9,   0,  66, 184, 180,  31,
         52, 141, 116,  70, 152,  98, 137, 114,  27,  59, 133,   4,  80,
         90, 137, 191,  21, 134,   7, 166, 122, 134

In [81]:
bag_model.estimators_[1]

DecisionTreeRegressor(random_state=1113451934)

In [82]:
y_pred = bag_model.predict(X_test)

In [83]:
np.sqrt(mean_squared_error(y_test , y_pred))

1934.0797031235577

## Model Tuning

In [88]:
bag_params = {"n_estimators": range(2,20)}

In [89]:
bag_cv_model = GridSearchCV(bag_model, bag_params, cv = 10)

In [91]:
bag_cv_model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=BaggingRegressor(bootstrap_features=True),
             param_grid={'n_estimators': range(2, 20)})

In [92]:
bag_cv_model.best_params_

{'n_estimators': 17}

In [93]:
bag_tuned = BaggingRegressor( n_estimators = 14, random_state = 45)

In [94]:
bag_tuned.fit(X_train, y_train)

BaggingRegressor(n_estimators=14, random_state=45)

In [95]:
y_pred = bag_tuned.predict(X_test)

In [96]:
np.sqrt(mean_squared_error(y_test, y_pred))

1893.7187253707032

# Random Forests

In [98]:
rf_model = RandomForestRegressor(random_state = 42)

In [99]:
rf_model.fit(X_train , y_train)

RandomForestRegressor(random_state=42)

In [100]:
y_pred = rf_model.predict(X_test)

In [102]:
np.sqrt(mean_squared_error(y_test , y_pred))

1801.9776487588415

## Model Tuning

In [103]:
rf_params = {'max_depth': list(range(1,10)),
            'max_features': [3,5,10,15],
            'n_estimators' : [100, 200, 500, 1000, 2000]}

In [104]:
rf_model = RandomForestRegressor(random_state = 42)

In [105]:
rf_cv_model = GridSearchCV(rf_model, rf_params, cv = 10, n_jobs = -1)

In [106]:
rf_cv_model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'max_features': [3, 5, 10, 15],
                         'n_estimators': [100, 200, 500, 1000, 2000]})

In [107]:
rf_cv_model.best_params_

{'max_depth': 9, 'max_features': 15, 'n_estimators': 100}

In [108]:
rf_tuned = RandomForestRegressor(max_depth = 9 , max_features = 15 , n_estimators = 100)

In [109]:
rf_tuned.fit(X_train , y_train)

RandomForestRegressor(max_depth=9, max_features=15)

In [110]:
y_pred = rf_tuned.predict(X_test)

In [111]:
np.sqrt(mean_squared_error(y_test , y_pred))

1917.7882910526764

# Gradient Boosting Machines

In [119]:
from sklearn.ensemble import GradientBoostingRegressor

In [120]:
gbm_model = GradientBoostingRegressor()
gbm_model.fit(X_train , y_train)

GradientBoostingRegressor()

In [121]:
y_pred = gbm_model.predict(X_test)

In [122]:
np.sqrt(mean_squared_error(y_test , y_pred))

1758.8339154913826

## Model Tuning

In [123]:
gbm_params = {
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'max_depth': [3, 5, 8,50,100],
    'n_estimators': [200, 500, 1000, 2000],
    'subsample': [1,0.5,0.75],
}

In [124]:
gbm = GradientBoostingRegressor()
gbm_cv_model = GridSearchCV(gbm, gbm_params, cv = 10, n_jobs = -1, verbose = 2)
gbm_cv_model.fit(X_train, y_train)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   53.8s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 19.3min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 27.4min
[Parallel(n_jobs=-1)]: Done 2400 out of 2400 | elapsed: 35.3min finished


GridSearchCV(cv=10, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 8, 50, 100],
                         'n_estimators': [200, 500, 1000, 2000],
                         'subsample': [1, 0.5, 0.75]},
             verbose=2)

In [125]:
gbm_cv_model.best_params_

{'learning_rate': 0.2, 'max_depth': 50, 'n_estimators': 200, 'subsample': 0.5}

In [126]:
gbm_tuned = GradientBoostingRegressor(learning_rate = 0.2,  
                                      max_depth = 50, 
                                      n_estimators = 200, 
                                      subsample = 0.5)

gbm_tuned = gbm_tuned.fit(X_train,y_train)

In [127]:
y_pred = gbm_tuned.predict(X_test)

In [128]:
np.sqrt(mean_squared_error(y_test , y_pred))

1765.6629757072803

# XGBoost

In [132]:
!pip install xgboost

In [133]:
import xgboost as xgb

In [134]:
DM_train = xgb.DMatrix(data = X_train, label = y_train)
DM_test = xgb.DMatrix(data = X_test, label = y_test)

In [135]:
from xgboost import XGBRegressor

In [136]:
xgb_model = XGBRegressor().fit(X_train, y_train)

In [137]:
y_pred = xgb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

2045.588341347811

## Model Tuning

In [138]:
xgb_grid = {
     'colsample_bytree': [0.4, 0.5,0.6,0.9,1], 
     'n_estimators':[100, 200, 500, 1000],
     'max_depth': [2,3,4,5,6],
     'learning_rate': [0.1, 0.01, 0.5]
}

In [139]:
xgb = XGBRegressor()

xgb_cv = GridSearchCV(xgb, 
                      param_grid = xgb_grid, 
                      cv = 10, 
                      n_jobs = -1,
                      verbose = 2)


xgb_cv.fit(X_train, y_train)

Fitting 10 folds for each of 300 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 21.6min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed: 27.7min finished


GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None,...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estima

In [140]:
xgb_cv.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 100}

In [141]:
xgb_tuned = XGBRegressor(colsample_bytree = 0.4, 
                         learning_rate = 0.1, 
                         max_depth = 3, 
                         n_estimators = 100) 

xgb_tuned = xgb_tuned.fit(X_train,y_train)

In [142]:
y_pred = xgb_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

1913.2596194380835

# Light GBM

In [143]:
!pip install lightgbm

In [144]:
from lightgbm import LGBMRegressor

In [145]:
lgbm = LGBMRegressor()
lgbm_model = lgbm.fit(X_train, y_train)

In [146]:
y_pred = lgbm_model.predict(X_test, 
                            num_iteration = lgbm_model.best_iteration_)

In [147]:
np.sqrt(mean_squared_error(y_test, y_pred))

2437.963529996202

# Model Tuning

In [148]:
lgbm_grid = {
    'colsample_bytree': [0.4, 0.5,0.6,0.9,1],
    'learning_rate': [0.01, 0.1, 0.5,1],
    'n_estimators': [20, 40, 100, 200, 500,1000],
    'max_depth': [1,2,3,4,5,6,7,8] }

lgbm = LGBMRegressor()
lgbm_cv_model = GridSearchCV(lgbm, lgbm_grid, cv=10, n_jobs = -1, verbose = 2)

In [149]:
lgbm_cv_model.fit(X_train, y_train)

Fitting 10 folds for each of 960 candidates, totalling 9600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done 1092 tasks      | elapsed:   39.6s
[Parallel(n_jobs=-1)]: Done 2224 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 3524 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 5304 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 7412 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 9600 out of 9600 | elapsed:  5.3min finished


GridSearchCV(cv=10, estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'colsample_bytree': [0.4, 0.5, 0.6, 0.9, 1],
                         'learning_rate': [0.01, 0.1, 0.5, 1],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8],
                         'n_estimators': [20, 40, 100, 200, 500, 1000]},
             verbose=2)

In [150]:
lgbm_cv_model.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.1,
 'max_depth': 5,
 'n_estimators': 1000}

In [151]:
lgbm_tuned = LGBMRegressor(learning_rate = 0.1, 
                           max_depth = 5, 
                           n_estimators = 1000,
                          colsample_bytree = 0.4)

lgbm_tuned = lgbm_tuned.fit(X_train,y_train)

In [152]:
y_pred = lgbm_tuned.predict(X_test)

In [153]:
np.sqrt(mean_squared_error(y_test, y_pred))

2109.530066981946